In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/14 00:07:12 WARN Utils: Your hostname, NivethanAsus resolves to a loopback address: 127.0.1.1; using 172.20.150.32 instead (on interface eth0)
23/09/14 00:07:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/14 00:07:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/14 00:07:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
consumers = spark.read.parquet("../../../data/insights/pre_insights/consumers.parquet/")

In [4]:
consumers_aggregated = consumers.groupBy("merchant_abn").agg(
                        f.countDistinct("consumer_id").alias("number_of_consumers"),
                        f.avg("consumer_fraud_probability_%").alias("average_consumer_fraud_probability"))

In [5]:
repeat_consumers = consumers.groupBy("merchant_abn", "consumer_id").agg(f.count("consumer_id").alias("consumer_order_times"))
repeat_consumers = repeat_consumers.filter(repeat_consumers.consumer_order_times > 1)
repeat_consumers_count = repeat_consumers.groupBy("merchant_abn").agg(f.count("consumer_order_times").alias("number_of_repeat_consumers"))

In [6]:
consumers_aggregated = consumers_aggregated.join(repeat_consumers_count, on = "merchant_abn", how = "leftouter")

In [7]:
consumers_aggregated.orderBy(f.col("number_of_consumers").desc(),
                             f.col("average_consumer_fraud_probability").asc(),
                             f.col("number_of_repeat_consumers").desc()).show()

+------------+-------------------+----------------------------------+--------------------------+
|merchant_abn|number_of_consumers|average_consumer_fraud_probability|number_of_repeat_consumers|
+------------+-------------------+----------------------------------+--------------------------+
| 45629217853|              20115|              0.042364532680646334|                     20095|
| 64203420245|              20115|               0.04343806405661244|                     20112|
| 86578477987|              20115|              0.043976018822537884|                     20113|
| 24852446429|              20114|               0.04446225413687606|                     20112|
| 89726005175|              20114|               0.04447215320508907|                     20090|
| 49891706470|              20113|               0.04247220417947379|                     20107|
| 46804135891|              20113|               0.04336435521125033|                     20102|
| 80324045558|              20

In [8]:
consumers_aggregated.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/consumers_agg.parquet")

In [9]:
spark.stop()